In [1]:
import os
import boto3
import pandas as pd
import pymysql  # For MySQL RDS

ModuleNotFoundError: No module named 'boto3'

In [ ]:

def lambda_handler(event, context):
    # Retrieve S3 bucket and object key from the event
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']

    # Download the file from S3
    s3_client = boto3.client('s3')
    local_file_path = '/tmp/' + os.path.basename(key)
    s3_client.download_file(bucket, key, local_file_path)

    # Clean the data (example: replace 'x' with 'y')
    cleaned_data = clean_data(local_file_path)

    # Connect to RDS and insert cleaned data
    db_connection = pymysql.connect(host='cardio-db.crkc44qquibr.us-east-1.rds.amazonaws.com',
                                   user='admin',
                                   password='rootroot',
                                   database='cardio-db')
    cursor = db_connection.cursor()

    # Example: Insert cleaned data into a table
    cursor.execute("INSERT INTO your_table (column1, column2) VALUES (%s, %s)", (cleaned_data['col1'], cleaned_data['col2']))

    db_connection.commit()
    db_connection.close()



ModuleNotFoundError: No module named 'boto3'

In [ ]:
def clean_data(data):
    # Handling missing values (dropping rows with missing values for simplicity)
    data = data.dropna()

    # Checking and handling duplicates
    data = data.drop_duplicates()

    # Correcting data types
    data['age'] = pd.to_numeric(data['age'], errors='coerce')
    data['height'] = pd.to_numeric(data['height'], errors='coerce')
    data['weight'] = pd.to_numeric(data['weight'], errors='coerce')

    # Encoding categorical variables
    data['gender'] = data['gender'].map({1: 'Male', 2: 'Female'})
    data['cholesterol'] = data['cholesterol'].map({1: 'Normal', 2: 'Above Normal', 3: 'High'})
    data['gluc'] = data['gluc'].map({1: 'Normal', 2: 'Above Normal', 3: 'High'})

    return data
